## My own resnet18 train on ood stanford car dataset

In [29]:
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision.models as models

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_dir = "/Data/federated_learning/large_vlm_distillation_ood/Resnet18_classification/car_data/car_data/"

In [31]:
train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


## Loading the model

In [32]:
dataset = torchvision.datasets.ImageFolder(root=dataset_dir+"train", transform = train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir+"test", transform = test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 16, shuffle=False, num_workers = 2)

# Load the pre-trained ResNet18 model
resnet18 = models.resnet18(pretrained=True)

# Modify the final fully connected layer
num_classes = 196
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, num_classes)
resnet18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Training:

In [33]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def train_model(train_loader, val_loader, model, criterion, num_epochs, lr, optimizer_class):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = optimizer_class(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005, nesterov=True)
    max_train_acc = 0.0
    max_val_acc = 0.0

    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        train_losses = []
        train_accs = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to the correct device
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_losses.append(loss.item())
            acc = accuracy(outputs, labels)
            train_accs.append(acc.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_losses)
        train_acc = np.mean(train_accs)
        max_train_acc = max(max_train_acc, train_acc)

        # Validation Phase
        model.eval()
        val_losses = []
        val_accs = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move data to the correct device
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())
                acc = accuracy(outputs, labels)
                val_accs.append(acc.item())

        val_loss = np.mean(val_losses)
        val_acc = np.mean(val_accs)
        max_val_acc = max(max_val_acc, val_acc)

        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, Train Acc: {100*(train_acc):.2f}, % "
              f"Val Loss: {val_loss:.4f}, Val Acc: {100*(val_acc):.2f}, %")

    return max_train_acc, max_val_acc



In [34]:
criterion, num_epochs, lr, optimizer = nn.CrossEntropyLoss(),25, 0.001, optim.SGD

In [35]:
max_train_acc, max_val_acc = train_model(trainloader, testloader, resnet18.to(device), criterion, num_epochs, lr, optimizer)

Epoch [1/25], Train Loss: 5.1324, Train Acc: 2.93, % Val Loss: 4.6384, Val Acc: 8.95, %
Epoch [2/25], Train Loss: 4.3107, Train Acc: 12.55, % Val Loss: 3.8060, Val Acc: 21.70, %
Epoch [3/25], Train Loss: 3.5761, Train Acc: 26.93, % Val Loss: 3.1030, Val Acc: 34.97, %
Epoch [4/25], Train Loss: 2.8977, Train Acc: 43.32, % Val Loss: 2.4779, Val Acc: 49.10, %
Epoch [5/25], Train Loss: 2.3357, Train Acc: 57.15, % Val Loss: 1.9775, Val Acc: 60.10, %
Epoch [6/25], Train Loss: 1.8678, Train Acc: 66.63, % Val Loss: 1.5944, Val Acc: 69.40, %
Epoch [7/25], Train Loss: 1.4869, Train Acc: 75.01, % Val Loss: 1.3322, Val Acc: 73.46, %
Epoch [8/25], Train Loss: 1.2110, Train Acc: 80.75, % Val Loss: 1.1256, Val Acc: 77.24, %
Epoch [9/25], Train Loss: 0.9936, Train Acc: 84.32, % Val Loss: 0.9708, Val Acc: 78.96, %
Epoch [10/25], Train Loss: 0.8280, Train Acc: 87.02, % Val Loss: 0.8629, Val Acc: 81.57, %
Epoch [11/25], Train Loss: 0.6832, Train Acc: 89.86, % Val Loss: 0.7720, Val Acc: 83.49, %
Epoch [12/